In [ ]:
import collections
from itertools import chain
import urllib.request as request
import pickle 

import numpy as np
import pandas as pd

import scipy.signal as signal
import scipy.special as special
import scipy.optimize as optimize

import matplotlib.pyplot as plt

import skimage.io
import skimage.transform
from skimage.restoration import estimate_sigma

import cv2
import math
import os

from libsvm import svmutil

import torch  
import torch.nn.functional as F
from PIL import Image, ImageStat

In [ ]:
def load(img_path):
    if isinstance(img_path, str):
        if os.path.exists(img_path):
            image = cv2.imread(img_path)
            grimage = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        else:
            raise FileNotFoundError('Image is not found on your system')
    else:
        raise ValueError('Only image can be passed to the constructor')
    return image, grimage

In [ ]:
def get_noise(img):
    H, W = img.shape
    M = [[1, -2, 1],
       [-2, 4, -2],
       [1, -2, 1]]

    sigma = np.sum(np.sum(np.absolute(signal.convolve2d(img, M))))
    sigma = sigma * math.sqrt(0.5 * math.pi) / (6 * (W-2) * (H-2))
    return sigma

In [ ]:
def get_contrast(img):
    contrast = img.std()
    return contrast

In [ ]:
def get_brightness(img):
    im = Image.open(img)
    stat = ImageStat.Stat(im)

    band = stat.mean
    r = band[0]
    g = band[1]
    b = band[2]
    return math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))

In [ ]:
image = None
Im = None
edgex, edgey = None, None

In [ ]:
def dom(Im):
    median_shift_up = np.pad(Im, ((0,2), (0,0)), 'constant')[2:,:]
    median_shift_down = np.pad(Im, ((2,0), (0,0)), 'constant')[:-2,:]
    domx = np.abs(median_shift_up - 2*Im + median_shift_down)

    median_shift_left = np.pad(Im, ((0,0), (0,2)), 'constant')[:,2:]
    median_shift_right = np.pad(Im, ((0,0), (2,0)), 'constant')[:,:-2]
    domy = np.abs(median_shift_left - 2*Im + median_shift_right)
    return domx, domy

In [ ]:
def contrast(Im):
        Cx = np.abs(Im - np.pad(Im, ((1,0), (0,0)), 'constant')[:-1, :])
        Cy = np.abs(Im - np.pad(Im, ((0,0), (1,0)), 'constant')[:, :-1])
        return Cx, Cy

In [ ]:
def smoothenImage(image):
    fil = np.array([0.5, 0, -0.5])
    image_smoothed = np.array([np.convolve(image[i], fil, mode="same") for i in range(image.shape[0])])

    image_smoothed = np.abs(image_smoothed)/(np.max(image_smoothed) + 1e-8)
    return image_smoothed

In [ ]:
def edges(image, edge_threshold=0.0001):
    smoothx = smoothenImage(image, transpose=True)
    smoothy = smoothenImage(image)
    global edgex
    edgex = smoothx > edge_threshold
    global edgey
    edgey = smoothy > edge_threshold

In [ ]:
def sharpness_matrix(Im, width=2):
    domx, domy = dom(Im)

    Cx, Cy = contrast(Im)

    Cx = np.multiply(Cx, edgex)
    Cy = np.multiply(Cy, edgey)

    Sx = np.zeros(domx.shape)
    Sy = np.zeros(domy.shape)

    for i in range(width, domx.shape[0]-width):
        num = np.abs(domx[i-width:i+width, :]).sum(axis=0)
        dn = Cx[i-width:i+width, :].sum(axis=0)
        Sx[i] = [(num[k]/dn[k] if dn[k] > 1e-3 else 0) for k in range(Sx.shape[1])]
    for j in range(width, domy.shape[1]-width):
        num = np.abs(domy[:, j-width: j+width]).sum(axis=1)
        dn = Cy[:, j-width:j+width].sum(axis=1)
        Sy[:, j] = [(num[k]/dn[k] if dn[k] > 1e-3 else 0) for k in range(Sy.shape[0])]
    return Sx, Sy

In [ ]:
def sharpness_measure(Im, width, sharpness_threshold, epsilon = 1e-8):
    Sx, Sy = sharpness_matrix(Im, width=width)
    Sx = np.multiply(Sx, edgex)
    Sy = np.multiply(Sy, edgey)

    n_sharpx = np.sum(Sx >= sharpness_threshold)
    n_sharpy = np.sum(Sy >= sharpness_threshold)

    n_edgex = np.sum(edgex)
    n_edgey = np.sum(edgey)

    Rx = n_sharpx/(n_edgex + 1e-8)
    Ry = n_sharpy/(n_edgey + 1e-8)

    S = np.sqrt(Rx**2 + Ry**2)
    return S

In [ ]:
def get_sharpness(img, width=2, sharpness_threshold=2, edge_threshold=0.0001, debug=False):
    Im = cv2.medianBlur(img, 3, cv2.CV_64F).astype("double")/255.0
    edges(img, edge_threshold=edge_threshold)
    score = sharpness_measure(Im, width=width, sharpness_threshold=sharpness_threshold)
    return score

In [ ]:
def normalize_kernel(kernel):
    return kernel / np.sum(kernel)

def gaussian_kernel2d(n, sigma):
    Y, X = np.indices((n, n)) - int(n/2)
    gaussian_kernel = 1 / (2 * np.pi * sigma ** 2) * np.exp(-(X ** 2 + Y ** 2) / (2 * sigma ** 2)) 
    return normalize_kernel(gaussian_kernel)

In [ ]:
def local_deviation(image, local_mean, kernel):
    sigma = image ** 2
    sigma = signal.convolve2d(sigma, kernel, 'same')
    return np.sqrt(np.abs(local_mean ** 2 - sigma))

In [ ]:
def calculate_mscn_coefficients(image, kernel_size=6, sigma=7/6):
    C = 1/255
    kernel = gaussian_kernel2d(kernel_size, sigma=sigma)
    local_mean = signal.convolve2d(image, kernel, 'same')
    local_var = local_deviation(image, local_mean, kernel)
    return (image - local_mean) / (local_var + C)

In [ ]:
def generalized_gaussian_dist(x, alpha, sigma):
    beta = sigma * np.sqrt(special.gamma(1 / alpha) / special.gamma(3 / alpha))
    coefficient = alpha / (2 * beta() * special.gamma(1 / alpha))
    return coefficient * np.exp(-(np.abs(x) / beta) ** alpha)

In [ ]:
def calculate_pair_product_coefficients(mscn_coefficients):
    return collections.OrderedDict({
        'mscn': mscn_coefficients,
        'horizontal': mscn_coefficients[:, :-1] * mscn_coefficients[:, 1:],
        'vertical': mscn_coefficients[:-1, :] * mscn_coefficients[1:, :],
        'main_diagonal': mscn_coefficients[:-1, :-1] * mscn_coefficients[1:, 1:],
        'secondary_diagonal': mscn_coefficients[1:, :-1] * mscn_coefficients[:-1, 1:]
    })

In [ ]:
def asymmetric_generalized_gaussian(x, nu, sigma_l, sigma_r):
    def beta(sigma):
        return sigma * np.sqrt(special.gamma(1 / nu) / special.gamma(3 / nu))
    
    coefficient = nu / ((beta(sigma_l) + beta(sigma_r)) * special.gamma(1 / nu))
    f = lambda x, sigma: coefficient * np.exp(-(x / beta(sigma)) ** nu)
        
    return np.where(x < 0, f(-x, sigma_l), f(x, sigma_r))

In [ ]:
def asymmetric_generalized_gaussian_fit(x):
    def estimate_phi(alpha):
        numerator = special.gamma(2 / alpha) ** 2
        denominator = special.gamma(1 / alpha) * special.gamma(3 / alpha)
        return numerator / denominator

    def estimate_r_hat(x):
        size = np.prod(x.shape)
        return (np.sum(np.abs(x)) / size) ** 2 / (np.sum(x ** 2) / size)

    def estimate_R_hat(r_hat, gamma):
        numerator = (gamma ** 3 + 1) * (gamma + 1)
        denominator = (gamma ** 2 + 1) ** 2
        return r_hat * numerator / denominator

    def mean_squares_sum(x, filter = lambda z: z == z):
        filtered_values = x[filter(x)]
        squares_sum = np.sum(filtered_values ** 2)
        return squares_sum / ((filtered_values.shape))

    def estimate_gamma(x):
        left_squares = mean_squares_sum(x, lambda z: z < 0)
        right_squares = mean_squares_sum(x, lambda z: z >= 0)

        return np.sqrt(left_squares) / np.sqrt(right_squares)

    def estimate_alpha(x):
        r_hat = estimate_r_hat(x)
        gamma = estimate_gamma(x)
        R_hat = estimate_R_hat(r_hat, gamma)

        solution = optimize.root(lambda z: estimate_phi(z) - R_hat, [0.2]).x

        return solution[0]

    def estimate_sigma(x, alpha, filter = lambda z: z < 0):
        return np.sqrt(mean_squares_sum(x, filter))
    
    def estimate_mean(alpha, sigma_l, sigma_r):
        return (sigma_r - sigma_l) * constant * (special.gamma(2 / alpha) / special.gamma(1 / alpha))
    
    alpha = estimate_alpha(x)
    sigma_l = estimate_sigma(x, alpha, lambda z: z < 0)
    sigma_r = estimate_sigma(x, alpha, lambda z: z >= 0)
    
    constant = np.sqrt(special.gamma(1 / alpha) / special.gamma(3 / alpha))
    mean = estimate_mean(alpha, sigma_l, sigma_r)
    
    return alpha, mean, sigma_l, sigma_r

In [ ]:
def calculate_brisque_features(image, kernel_size=7, sigma=7/6):
    def calculate_features(coefficients_name, coefficients, accum=np.array([])):
        alpha, mean, sigma_l, sigma_r = asymmetric_generalized_gaussian_fit(coefficients)

        if coefficients_name == 'mscn':
            var = (sigma_l ** 2 + sigma_r ** 2) / 2
            return [alpha, var]
        
        return [alpha, mean, sigma_l ** 2, sigma_r ** 2]
    
    mscn_coefficients = calculate_mscn_coefficients(image, kernel_size, sigma)
    coefficients = calculate_pair_product_coefficients(mscn_coefficients)
    
    features = [calculate_features(name, coeff) for name, coeff in coefficients.items()]
    flatten_features = list(chain.from_iterable(features))
    return np.array(flatten_features, dtype=object)

In [ ]:
def scale_features(features):
    with open('normalize.pickle', 'rb') as handle:
        scale_params = pickle.load(handle)
    
    min_ = np.array(scale_params['min_'])
    max_ = np.array(scale_params['max_'])
    
    return -1 + (2.0 / (max_ - min_) * (features - min_))

def calculate_image_quality_score(brisque_features):
    model = svmutil.svm_load_model('brisque_svm.txt')
    scaled_brisque_features = scale_features(brisque_features)
    
    x, idx = svmutil.gen_svm_nodearray(
        scaled_brisque_features,
        isKernel=(model.param.kernel_type == svmutil.PRECOMPUTED))
    
    nr_classifier = 1
    prob_estimates = (svmutil.c_double * nr_classifier)()
    
    return svmutil.libsvm.svm_predict_probability(model, x, prob_estimates)

In [ ]:
def get_brisque(img):
    image = skimage.io.imread(img)
    gray_image = skimage.color.rgb2gray(image)
    mscn_coefficients = calculate_mscn_coefficients(gray_image, 7, 7/6)
    coefficients = calculate_pair_product_coefficients(mscn_coefficients)
    brisque_features = calculate_brisque_features(gray_image, kernel_size=7, sigma=7/6)
    downscaled_image = cv2.resize(gray_image, None, fx=1/2, fy=1/2, interpolation = cv2.INTER_CUBIC)
    downscale_brisque_features = calculate_brisque_features(downscaled_image, kernel_size=7, sigma=7/6)
    brisque_features = np.concatenate((brisque_features, downscale_brisque_features))
    return calculate_image_quality_score(brisque_features)

In [ ]:
def gaussian(window_size, sigma):
    gauss =  torch.Tensor([math.exp(-(x - window_size//2)**2/float(2*sigma**2)) for x in range(window_size)])
    return gauss/gauss.sum()

In [ ]:
def create_window(window_size, channel=1):
    _1d_window = gaussian(window_size=window_size, sigma=1.5).unsqueeze(1)
    _2d_window = _1d_window.mm(_1d_window.t()).float().unsqueeze(0).unsqueeze(0)
    window = torch.Tensor(_2d_window.expand(channel, 1, window_size, window_size).contiguous())
    return window

In [ ]:
def ssim(img1, img2, val_range, window_size=11, window=None):
    L = val_range
    pad = window_size // 2
    
    try:
        _, channels, height, width = img1.size()
    except:
        channels, height, width = img1.size()

    if window is None: 
        real_size = min(window_size, height, width)
        window = create_window(real_size, channel=channels).to(img1.device)

    mu1 = F.conv2d(img1, window, padding=pad, groups=channels)
    mu2 = F.conv2d(img2, window, padding=pad, groups=channels)
    mu1_sq = mu1 ** 2
    mu2_sq = mu2 ** 2 
    mu12 = mu1 * mu2
    sigma1_sq = F.conv2d(img1 * img1, window, padding=pad, groups=channels) - mu1_sq
    sigma2_sq = F.conv2d(img2 * img2, window, padding=pad, groups=channels) - mu2_sq
    sigma12 =  F.conv2d(img1 * img2, window, padding=pad, groups=channels) - mu12
    C1 = (0.01 ) ** 2
    C2 = (0.03 ) ** 2 

    contrast_metric = (2.0 * sigma12 + C2) / (sigma1_sq + sigma2_sq + C2)
    contrast_metric = torch.mean(contrast_metric)
    numerator1 = 2 * mu12 + C1  
    numerator2 = 2 * sigma12 + C2
    denominator1 = mu1_sq + mu2_sq + C1 
    denominator2 = sigma1_sq + sigma2_sq + C2

    ssim_score = (numerator1 * numerator2) / (denominator1 * denominator2)
    return ssim_score.mean() 

In [ ]:
def get_SSIM(img1, img2):
    load_images = lambda x: np.asarray(Image.open(x).resize((480, 640)))
    tensorify = lambda x: torch.Tensor(x.transpose((2, 0, 1))).unsqueeze(0).float().div(255.0)
    img1 = load_images(img1)
    img2 = load_images(img2)
    gauss_dis = gaussian(11, 1.5)
    window = create_window(11, 3)
    timg1 = tensorify(img1)
    timg2 = tensorify(img2)
    score = ssim(_img1, _img2, val_range=255)
    point = float(str(gg)[7:-1])
    return point

In [ ]:
def get_dynamic_range(img):
    dynamic_range = []
    image = cv2.imread(img, cv2.IMREAD_COLOR)
    pixel_brightness = []
    for x in range (1,480):
        for y in range (1,640):
            try:
                pixel = image[x,y]
                R, G, B = pixel
                brightness = sum([R,G,B])/3
                pixel_brightness.append(brightness)
            except IndexError:
                pass
    dyn_range = round(np.log2(max(pixel_brightness))-np.log2(min((pixel_brightness))), 2)
    dynamic_range.append(dyn_range)
    return dynamic_range

In [ ]:
def IQA(img_path, img2_path=False):
    if img2_path:
        colomns = ['SSIM']
        df_scale = pd.DataFrame(index=range(1), columns=colomns)
        df_interp = pd.DataFrame(index=range(1), columns=colomns)
        im1 = load(img_path)
        im2 = load(img2_path)
        df_scale['SSIM'][0] = get_SSIM(img_path, img2_path)
        if df_scale['SSIM'][0] == 1:
            df_interp['SSIM'][0] = 'Exact same'
        elif df_scale['SSIM'][0] > 0.8:
            df_interp['SSIM'][0] = 'Very similar'
        elif df_scale['SSIM'][0] > 0.5:
            df_interp['SSIM'][0] = 'Similar'
        else:
            df_interp['SSIM'][0] = 'Not similar'
        
        return df_scale, df_interp
    
    else:
        colomns = ['Noise', 'Brightness', 'Contrast', 'Sharpness', 'BRISQUE', 'Dynamic Range']
        df_scale = pd.DataFrame(index=range(1), columns=colomns)
        df_interp = pd.DataFrame(index=range(1), columns=colomns)

        origimg, greyimg = load(img_path)

        df_scale['Noise'][0] = get_noise(grimg)
        if df_scale['Noise'][0] > 10:
            df_interp['Noise'][0] = 'Very noisy'
        elif df_scale['Noise'][0] > 5:
            df_interp['Noise'][0] = 'Noisy'
        else:
            df_interp['Noise'][0] = 'Non-noisy'

        df_scale['Contrast'][0] = get_contrast(grimg)
        if df_scale['Contrast'][0] > 40:
            df_interp['Contrast'][0] = 'High contrast'
        else:
            df_interp['Contrast'][0] = 'Low contrast'

        df_scale['Brightness'][0] = get_brightness(img_path)
        if df_scale['Brightness'][0] > 200:
            df_interp['Brightness'][0] = 'Very high brightness'
        elif df_scale['Brightness'][0] > 120:
            df_interp['Brightness'][0] = 'High brightness'
        elif df_scale['Brightness'][0] > 20:
            df_interp['Brightness'][0] = 'Low brightness'
        else:
            df_interp['Brightness'][0] = 'Very low brightness'

        df_scale['Sharpness'][0] = get_sharpness(grimg)
        if df_scale['Sharpness'][0] > 1:
            df_interp['Sharpness'][0] = 'Very sharp'
        elif df_scale['Sharpness'][0] > math.sqrt(2)/2:
            df_interp['Sharpness'][0] = 'Sharp'
        elif df_scale['Sharpness'][0] > 0.3:
            df_interp['Sharpness'][0] = 'Unsharp'
        else:
            df_interp['Sharpness'][0] = 'Very unsharp'

        df_scale['BRISQUE'][0] = get_brisque(img_path)
        if df_scale['BRISQUE'][0] > 80:
            df_interp['BRISQUE'][0] = 'Very good'
        elif df_scale['BRISQUE'][0] > 50:
            df_interp['BRISQUE'][0] = 'Good'
        elif df_scale['BRISQUE'][0] > 20:
            df_interp['BRISQUE'][0] = 'Bad'
        else:
            df_interp['BRISQUE'][0] = 'Very bad'
            
        df_scale['Dynamic Range'][0] = get_dynamic_range(img_path)
        if df_scale['Dynamic Range'][0] > 14:
            df_interp['Dynamic Range'][0] = 'Full sunlight'
        elif df_scale['Dynamic Range'][0] > 12:
            df_interp['Dynamic Range'][0] = 'Cloudy'
        else:
            df_interp['Dynamic Range'][0] = 'Deep shade'

        return df_scale, df_interp